In [ ]:
import matplotlib
import pandas as pd
import numpy as np
import pickle
import urllib.request
import dateutil.parser
import dateutil.rrule
import dateutil.tz
import datetime
import warnings
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
import math
import re

# Plotting is externalised
import importlib
import traffic

importlib.reload(traffic)

matplotlib.rcParams.update({
    'font.size': 13,
    'timezone': 'Europe/London',
    'figure.max_open_warning': False
})

In [ ]:
tzLocal = dateutil.tz.gettz('Europe/London')
tzUTC = dateutil.tz.gettz('UTC')

In [ ]:
resampleInterval = 60 * 8  # N minute data

def prepareTotal(pdInput, interval = 60 * 8):
    return pdInput \
        .resample('%us' % interval, closed='right', label='right') \
        .sum() \
        .sum(axis=1) \
        .to_frame() \
        .rename(columns={0: 'Total'}) \
        .replace({0: np.nan})

# Traffic volumes across Tyne and Wear against profile

The below charts show the last four weeks of daily traffic profiles across the whole of Tyne and Wear. This is based on plate reads by the region's ANPR network, and is normalised against the normal peak daily vehicle flows (i.e. 100% is normal worst part of rush hour traffic).

You can [download the underlying counts as a CSV file](https://covid.view.urbanobservatory.ac.uk/output/t%26w-anpr-volumes-pd-16min.csv) and [accompanying metadata](https://covid.view.urbanobservatory.ac.uk/output/t%26w-anpr-volumes-point-metadata-pd.csv), or access it through [an API](https://api.newcastle.urbanobservatory.ac.uk/api/v2/sensors/entity?metric=%22of%20link%22&page=1).

In [ ]:
pdTrafficTyneWear = pickle.load(open('../cache/recent-traffic-volumes-pd.pkl', 'rb'))

print('Last data obtained %s' 
    % (np.max(pdTrafficTyneWear.index).strftime('%d %B %Y %H:%M')))

In [ ]:
pdTrafficAllSum = prepareTotal(pdTrafficTyneWear, resampleInterval)

#pdTrafficAllSum.plot(figsize=(18, 8))
#pdTrafficAllSum

In [ ]:
#pdTrafficAllSum.plot(figsize=(18, 8))

In [ ]:
dailyProfiles = traffic.generateDailyProfiles(pdTrafficAllSum, resampleInterval)

In [ ]:
for p in dailyProfiles['profile'].keys():
    if not isinstance(dailyProfiles['profile'][p], pd.DataFrame):
        continue
        
    daysToFix = []
    for day in list(dailyProfiles['profile'][p]['Day of week'].unique()):
        if min(dailyProfiles['profile'][p][dailyProfiles['profile'][p]['Day of week'] == day].index.to_series()).minute > 0:
            daysToFix.append(day)
    
    dailyProfiles['profile'][p].index = dailyProfiles['profile'][p].index.to_series().apply(lambda t: t - pd.Timedelta(minutes=56) if t.strftime('%A') in daysToFix else t)

In [ ]:
# For Twitter sharing... transparency doesn't work so well when some browsers default to black background
%config InlineBackend.print_figure_kwargs={'facecolor' : '#fafafa'}

fig, axs = traffic.plotDailyProfile(dailyProfiles)

In [ ]:
#pdPointMeta = pickle.load(open('../cache/recent-traffic-volumes-point-metadata-pd.pkl', 'rb'))
#highways = pdPointMeta['highwayDescription'].apply(lambda hd: re.sub(' \(.*\)$', '', hd)).unique()

In [ ]:
%config InlineBackend.print_figure_kwargs={'facecolor' : '#fafafa'}

# Uncomment to plot every road...

i="""
individualRoadResample = resampleInterval * 2
for c in highways:
    matchingColumns = list(filter(
        lambda cn: cn in pdTrafficTyneWear.columns,
        list(pdPointMeta[pdPointMeta['highwayDescription'].apply(lambda hd: c + ' (' in hd)]['timeseriesName'])
    ))
    if len(matchingColumns) <= 0:
        continue
        
    highwayFeeds = pdTrafficTyneWear[matchingColumns]
    
    pdTrafficSingleRoad = prepareTotal(highwayFeeds.resample('%us' % individualRoadResample).sum(), individualRoadResample)
    dailyProfiles = traffic.generateDailyProfiles(pdTrafficSingleRoad, individualRoadResample)
    # pointMeta = pdPointMeta[pdPointMeta['timeseriesName'] == c].to_dict(orient='records')[0]
    fig, axs = traffic.plotDailyProfile(dailyProfiles, title=c)
"""